In [1]:
import sys
from pathlib import Path
path = Path("C:/Users/shaur/Desktop/Glean_Implementation")
sys.path.append(str(path))

In [2]:
import re
import os

import json
from utils import config
import importlib
importlib.reload(config)

<module 'utils.config' from 'C:\\Users\\shaur\\Desktop\\Glean_Implementation\\utils\\config.py'>

In [3]:
def get_reg_nums(all_words):
    reg_nums = []
    reg_no_re = r'^[0-9a-zA-Z-:]+$'
    for word in all_words:
        if not re.search('\d', word['text']):
            continue
        if len(word['text']) < 4:
            continue
        result = re.findall(reg_no_re, word['text'])
        if result:
            reg_nums.append({
                'text': word['text'],
                'x1': word['x1'],
                'y1': word['y1'],
                'x2': word['x2'],
                'y2': word['y2']
            })

    return reg_nums

In [4]:
# def get_reg_names(data):
#     reg_names = []

#     for para in data['ocr']['pages'][0]['paragraphs']:


In [5]:
def get_candidates(data):
        all_words = []  

        element_width = int(data['ocr']['pages'][0]['dimension']['width'])
        element_height = int(data['ocr']['pages'][0]['dimension']['height'])

        for token in data['ocr']['pages'][0]['tokens']:
            if token['text'].strip() != "":
                all_words.append({
                    'text': token['text'],
                    'x1': int(round(float(token['bbox'][1])*element_width)),
                    'y1': int(round(float(token['bbox'][2])*element_height)),
                    'x2': int(round(float(token['bbox'][3])*element_width)),
                    'y2': int(round(float(token['bbox'][4])*element_height))})
        text = ' '.join([word['text'].strip() for word in all_words])

        try:
            reg_num_candidates = get_reg_nums(all_words)
        except Exception as e:
            reg_num_candidates = []
        # try:
        #     reg_name_candidates = get_reg_names(data)
        # except Exception as e:
        #     reg_name_candidates = []

        candidate_data = {
            'registration_num': reg_num_candidates
            # 'registrant_name': reg_name_candidates
            # 'total': total_amount_candidates
        }
        return candidate_data

In [6]:
ocr_path = config.OCR_DIR
output_dir = config.CANDIDATE_DIR

os.makedirs(output_dir, exist_ok=True)

In [7]:
annotation_files = list(ocr_path.glob("*.json"))

for ann in annotation_files:
    with open(ann) as f:
        data = json.load(f)
        result = get_candidates(data)

        output_file_name = os.path.splitext(data['filename'])[0]
        output_file_path = os.path.join(output_dir, output_file_name+'.json')

        with open(output_file_path, 'w', encoding='utf-8') as output_file:
            json.dump(result, output_file, ensure_ascii=False, indent=2)
